In [25]:
! pip install  ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.2.0-py3-none-any.whl size=24770 sha256=43aba0f285c87b6e9aa9cf47b033d3b88e9e8bab7fc07140107837684e1a8c0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-hhmih76j/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.2.0
    Uninstalling boston311-0.2.0:
      Successfully uninstalled boston311-0.2.0


In [26]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import pandas as pd
import numpy as np
import pickle
from ast import literal_eval
from datetime import datetime, timedelta
from keras_tuner import HyperParameters
from boston311 import Boston311LogReg, Boston311KerasNN

In [27]:
today_datestring, tomorrow_datestring, thirty_days_ago_datestring = Boston311LogReg().get_datestrings()

In [28]:
KerasNN_model = Boston311KerasNN()

In [29]:
KerasNN_model.train_date_range={'start':'2023-01-01','end':thirty_days_ago_datestring}

KerasNN_model.predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring}

KerasNN_model.feature_columns=['queue', 'subject', 'reason', 'department']

KerasNN_model.scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']}, 'survivalTimeMin':300, 'survivalTimeFill':tomorrow_datestring}

KerasNN_model.epochs = 20
KerasNN_model.batch_size = 32

In [30]:

hour_interval = 48
max_days = 120

#KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_statistics(df, num_intervals=60)
KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_fixed_interval(hour_interval, max_days)

start_nodes = 256  
end_nodes = 128
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001

hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
bin_number = len(KerasNN_model.bin_edges) - 1
hp.Fixed('final_layer', bin_number)
hp.Fixed('final_activation', 'softmax')
KerasNN_model.best_hyperparameters = hp

In [31]:

'''

pickle_file = 'dataframe.pkl'
xz_file = '../dataframe.csv.xz'
#unpack xz file if it exists
if os.path.exists(xz_file):
    os.system(f'unxz {xz_file}')
    df = pd.read_csv('dataframe.csv')
    pickle.dump(df, open(pickle_file, "wb"))
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'

df = KerasNN_model.pkl_load_data(EXTRA_mydata_FILE, pickle_file)

# if df has a column service_request_id, do the following
if 'service_request_id' in df.columns:
    df.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)

    df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
    is_numeric = df['case_enquiry_id'].str.isnumeric()
    df = df[is_numeric]
    df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        df[col] = df[col].apply(literal_eval).apply(np.array)

    pickle.dump(df, open(pickle_file, "wb"))

# Assuming df is your DataFrame
column_names = ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']
df = KerasNN_model.flatten_and_replace_columns(df, column_names)
'''




'\n\npickle_file = \'dataframe.pkl\'\nxz_file = \'../dataframe.csv.xz\'\n#unpack xz file if it exists\nif os.path.exists(xz_file):\n    os.system(f\'unxz {xz_file}\')\n    df = pd.read_csv(\'dataframe.csv\')\n    pickle.dump(df, open(pickle_file, "wb"))\nEXTRA_mydata_FILE = \'./cls_and_pooled_embeddings_with_three_cols.csv\'\n\ndf = KerasNN_model.pkl_load_data(EXTRA_mydata_FILE, pickle_file)\n\n# if df has a column service_request_id, do the following\nif \'service_request_id\' in df.columns:\n    df.rename(columns={\'service_request_id\':\'case_enquiry_id\'}, inplace=True)\n\n    df[\'case_enquiry_id\'] = df[\'case_enquiry_id\'].astype(str)\n    is_numeric = df[\'case_enquiry_id\'].str.isnumeric()\n    df = df[is_numeric]\n    df[\'case_enquiry_id\'] = df[\'case_enquiry_id\'].astype(\'int64\')\n    for col in [\'desc_cls_embedding\', \'desc_pooled_embedding\', \'name_cls_embedding\', \'name_pooled_embedding\', \'code_cls_embedding\', \'code_pooled_embedding\']:\n        df[col] = df[c

In [32]:
case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
data = KerasNN_model.pkl_load_data(case_data_csv, case_data_file)

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:95: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [33]:
#get current datetime in Boston timezone as string
my_datetime_str = KerasNN_model.get_current_datetime_str()

In [34]:
KerasNN_model.run_pipeline(data=data)


bin_labels is None


Starting Training at 2024-01-01 15:39:42.303818
input_dim: 244
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 256)               62720     
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 61)                7869      
                                                                 
Total params: 103485 (404.24 KB)
Trainable params: 103485 (404.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
<class 'pandas.core.frame.DataFrame'> (202055, 61)
<class 'pandas.core.frame.DataFrame'> (50514, 61)
run fit

Epoch 1/20
6315/6315 [==============================] - 12s 2ms/step - loss: 1.7333 - accuracy: 0.59

0.6090192794799805

In [35]:
predictions = KerasNN_model.predict()

Checking files_dict
files_dict is None
trying to call get311URLs
trying to get csv URLs


Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmphdne3icr.csv for year 2024
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv for year 2023
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv for year 2022
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv for year 2021
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv for year 2020
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv for year 2019
Found URL:

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:336: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:336: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


columns in data before drop: Index(['case_enquiry_id', 'open_dt', 'sla_target_dt', 'closed_dt', 'on_time',
       'case_status', 'closure_reason', 'case_title', 'subject', 'reason',
       'type', 'queue', 'department', 'submitted_photo', 'closed_photo',
       'location', 'fire_district', 'pwd_district', 'city_council_district',
       'police_district', 'neighborhood', 'neighborhood_services_district',
       'ward', 'precinct', 'location_street_name', 'location_zipcode',
       'latitude', 'longitude', 'geom_4326', 'source', 'survival_time',
       'event', 'ward_number', 'survival_time_hours'],
      dtype='object')
columns to drop: Index(['case_status', 'case_title', 'city_council_district', 'closed_dt',
       'closed_photo', 'closure_reason', 'fire_district', 'geom_4326',
       'latitude', 'location', 'location_street_name', 'location_zipcode',
       'longitude', 'neighborhood', 'neighborhood_services_district',
       'on_time', 'open_dt', 'police_district', 'precinct', 'pwd_

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:267: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, fake_df], ignore_index=True)


588/588 [==============================] - 0s 543us/step


In [36]:
print(predictions)

(array([[9.8571086e-01, 1.1121310e-02, 1.2720719e-03, ..., 8.4666963e-06,
        8.4274643e-06, 3.3580855e-04],
       [2.7253813e-01, 9.7030632e-02, 6.1388515e-02, ..., 4.1984334e-03,
        2.9380131e-03, 1.3642371e-01],
       [1.4051642e-01, 2.3272300e-01, 1.8313232e-01, ..., 3.4495746e-04,
        1.8950374e-04, 3.7587893e-03],
       ...,
       [6.5039122e-01, 1.3639423e-01, 9.8013029e-02, ..., 1.6034235e-05,
        1.7299879e-05, 1.0340229e-04],
       [6.5039122e-01, 1.3639423e-01, 9.8013029e-02, ..., 1.6034235e-05,
        1.7299879e-05, 1.0340229e-04],
       [9.9110109e-01, 6.4599221e-03, 7.6955603e-04, ..., 7.9747961e-06,
        8.7662556e-06, 3.0992634e-04]], dtype=float32),         case_enquiry_id             open_dt        sla_target_dt  \
276736     101005187078 2023-12-02 06:19:00  2023-12-07 03:30:00   
276737     101005187122 2023-12-02 06:47:49  2023-12-04 06:47:49   
276739     101005187183 2023-12-02 07:31:07                  NaN   
276740     101005187139 20

In [37]:
#set model folder constant
MODEL_FOLDER = './daily_models'

def save_model_to_dir(model, folder_name, datetime_string=None):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    if datetime_string is None:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    else:
        timestamp = datetime_string
    
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)


save_model_to_dir(KerasNN_model, KerasNN_model.model_type, datetime_string=my_datetime_str)
